In [1]:
from datasets import load_dataset
import requests
import ciso8601
import time
import json
from datetime import datetime

/Users/michaelnath/.virtualenvs/224n_final_proj/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds = load_dataset("codeparrot/github-code", streaming=True, split='train', languages=["Python"])

Using custom data configuration Python-all-4b2efe4a27feed92


In [4]:
def construct_feature_set(code_entry) -> dict[str:int]:
    features = dict();
    user_repo_name = code_entry['repo_name'].split('/')
    owner = user_repo_name[0]
    repo = "".join(user_repo_name[1:])
    repo_response = requests.get(f"https://api.github.com/repos/{owner}/{repo}")
    content = json.loads(repo_response.text)
    features["num_stars"] = content.get("stargazers_count", 0)
    features["num_forks"] = content.get("forks_count", 0)
    features["num_watchers"] = content.get("watchers_count", 0)
    features["num_open_issues"] = content.get("open_issues_count", 0)
    parsed_datetime = ciso8601.parse_datetime(content.get("created_at", datetime.now().isoformat()))
    timestamp = time.mktime(parsed_datetime.timetuple())
    features["created_at"] = timestamp
    return features

In [5]:
# THE BELOW IS FOR PYTHON FILES (LEVERAGES INDENTATION RULES)
def construct_list_of_functions(code_entry) -> list[str]:
    raw_code = code_entry["code"]
    lines = raw_code.split('\n')
    start = -1
    functions = []
    amnt_tabs = 0
    for i in range(len(lines)):
        # disregard empty lines (prune trailing whitespace later)
        if (start != -1 and len(lines[i]) > 0):
            amnt_tabs_new = len(lines[i].rstrip()) - len(lines[i].strip())
            if amnt_tabs_new <= amnt_tabs:
                functions.append(("\n".join(lines[start:i])).strip())
                start = -1
        elif lines[i].lstrip().startswith("def "):
            start = i
            amnt_tabs = len(lines[i].rstrip()) - len(lines[i].strip())
    return functions


In [10]:
MAX_AMNT_CODE_ENTRIES = 1
augmented_code_entry_data = []
for i in range(MAX_AMNT_CODE_ENTRIES):
    code_entry = next(iterable_ds)
    features_set = construct_feature_set(code_entry)
    functions = construct_list_of_functions(code_entry)
    data = dict()
    # data["features"] = []
    # data["snippets"] = []
    augmented_code_entry_data.append(data)
print(augmented_code_entry_data[0])

{}


In [6]:
def augment_code_entry(entry):
    repo_features = construct_feature_set(entry)
    entry["reputation_features"] = repo_features
    if entry["language"] != "Python":
        return entry
    entry["functions"] = construct_list_of_functions(entry)
    return entry

In [7]:
augmented_dataset = ds.map(augment_code_entry)

In [9]:
dataset_heads = augmented_dataset.take(4)
arr = list(dataset_heads)
print(arr)

[{'code': 'from django import forms\nfrom django.core.exceptions import ValidationError\nfrom django.core.validators import validate_slug\nfrom django.db import models\nfrom django.utils import simplejson as json\nfrom django.utils.text import capfirst\nfrom django.utils.translation import ugettext_lazy as _\n\nfrom philo.forms.fields import JSONFormField\nfrom philo.utils.registry import RegistryIterator\nfrom philo.validators import TemplateValidator, json_validator\n#from philo.models.fields.entities import *\n\n\nclass TemplateField(models.TextField):\n\t"""A :class:`TextField` which is validated with a :class:`.TemplateValidator`. ``allow``, ``disallow``, and ``secure`` will be passed into the validator\'s construction."""\n\tdef __init__(self, allow=None, disallow=None, secure=True, *args, **kwargs):\n\t\tsuper(TemplateField, self).__init__(*args, **kwargs)\n\t\tself.validators.append(TemplateValidator(allow, disallow, secure))\n\n\nclass JSONDescriptor(object):\n\tdef __init__(s

In [21]:
print(arr[1]["repo_name"])
print(arr[1]["reputation_features"])
print("\n".join(arr[1]["functions"]))

nficano/python-lambda
{'num_stars': 1426, 'num_forks': 229, 'num_watchers': 1426, 'num_open_issues': 63, 'created_at': 1456485666.0}
def load_source(module_name, module_path):
    """Loads a python module from the path of the corresponding file."""

    if sys.version_info[0] == 3 and sys.version_info[1] >= 5:
        import importlib.util
        spec = importlib.util.spec_from_file_location(module_name, module_path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
    elif sys.version_info[0] == 3 and sys.version_info[1] < 5:
        import importlib.machinery
        loader = importlib.machinery.SourceFileLoader(module_name, module_path)
        module = loader.load_module()
    return module
def deploy(
    src,
    requirements=None,
    local_package=None,
    config_file="config.yaml",
    profile_name=None,
    preserve_vpc=False,
def deploy_s3(
    src,
    requirements=None,
    local_package=None,
    config_file="config.yaml",
 